In [10]:
from fastai.vision.all import *
import cv2

In [3]:
train_path = './commaai/data/train/'
valid_path = './commaai/data/valid/'
img_diff_path = './commaai/diff/'

In [47]:
# create a dataset based on img differences
# so img(2) - img(1) for imgs in sequence
# labes become speed(2) - speed(1)
# idea is that we learn only the differences
# as footage in sequence seems pretty identical


def extract_difference(img_dir, img_diff_path):
    images = list(sorted(os.listdir(img_dir))) # make sure you iterate in order
    
    if not os.path.exists(img_diff_path):
        os.makedirs(img_diff_path)
    looper = 0
    for img in images:
        try:
            img_1 = cv2.imread(train_path + img)

            next_image = cv2.imread(train_path + images[looper + 1])

            img_diff = next_image - img_1

            cv2.imwrite(img_diff_path + f'{looper}.jpg', img_diff)
            looper += 1
        except IndexError as e:
            pass

In [48]:
extract_difference(train_path, img_diff_path)

In [54]:
def create_labels(txt_path):
    with open(txt_path, 'r', encoding='ISO-8859-1') as fh:
        lister = fh.readlines()
        
    looper = 0
    differences = []
    for speed in lister:
        try:
            speed_diff = float(lister[looper + 1]) - float(speed)
            differences.append(speed_diff)
        except IndexError as e:
            pass
    return differences

In [55]:
labels = create_labels('./commaai/' + 'train.txt')

In [35]:
def get_speed(img_name):
    img_name = str(img_name)
    img_only = img_name.replace('commaai/diff/','')
    idx = int(img_only.replace('.jpg',''))
    return labels[idx]

In [38]:
idxs = [x for x in range(18000, 20398, 1)] # grab ~10% of images for val

In [57]:
speeder = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_items=get_image_files,
    get_y=get_speed,
    splitter=IndexSplitter(idxs),
    batch_tfms=[*aug_transforms(size=(240,320)), 
                Normalize.from_stats(*imagenet_stats)]
)

In [58]:
dls = speeder.dataloaders(img_diff_path)

In [59]:
learn = cnn_learner(dls, resnet34, metrics=rmse)
learn.loss = MSELossFlat()

In [60]:
learn.fine_tune(12)

epoch,train_loss,valid_loss,_rmse,time
0,73.067833,22.009066,4.691382,01:19


epoch,train_loss,valid_loss,_rmse,time
0,20.785078,8.447062,2.906383,01:41
1,15.264235,6.388496,2.527548,01:38
2,10.883669,4.014466,2.003613,01:42
3,7.952866,2.624630,1.620071,01:37
4,6.557484,2.234188,1.494720,01:39
5,5.554434,1.376750,1.173350,01:38
6,4.661615,1.229061,1.108630,01:37
7,4.105191,1.067537,1.033217,01:37
8,3.430081,0.803014,0.896111,01:37
9,3.673837,0.722111,0.849771,01:37
